<a href="https://colab.research.google.com/github/Matteo-Candi/ADM_HW3/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 3 - Places of the world**
---

In [20]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from datetime import datetime

## **1. df collection**

### 1.1. Get the list of places

In [2]:
with open('Places_url.txt', 'x') as f:
    for i in tqdm(range(1,401)):             
        url = f'https://www.atlasobscura.com/places?page={i}&sort=likes_count'
        result = requests.get(url)
        soup = bs(result.text)
        matches = soup.find_all("a", {"class": "content-card content-card-place"})
        for match in matches:
            f.write(match['href'] + '\n')

'c:\\Users\\yharr\\OneDrive\\Bureau\\DataScience\\ADM\\homeworks\\ADM_Homework3\\repo_git\\ADM_HW3../'

### 1.2. Crawl places

In [ ]:
os.mkdir(f'Locations')
places_url = pd.read_csv('Places_url.txt', header=None)


for page in tqdm(range(0,400)):
    os.mkdir(f'Locations/page_{page + 1}')
    for loc in range(0,18):
        with open(f'Locations/page_{page + 1}/location_{(18 * page) + loc + 1}.html', 'x', encoding="utf-8") as f:
            url = 'https://www.atlasobscura.com' + places_url.iloc[(18 * page) + loc][0]
            result = requests.get(url)
            html = result.text
            f.write(html)

### 1.3 Parse downloaded pages

In [113]:
def get_data_for_one_place(html_file):

    soup = bs(html_file)

    df = pd.DataFrame(columns=['placeName','placeTags','numPeopleVisited','numPeopleWant','placeDesc','placeShortDesc','placeNearby','placeAddress','placeAlt','placeLong','placeEditors','placePubDate','placeRelatedLists','placeRelatedPlaces','placeURL']) 

    placeName = soup.find('h1', {'class':'DDPage__header-title'})
    if placeName is not None :
        df['placeName'] = [placeName.text]
    else :
        df['placeName'] = ''

    placeTags = soup.find('div', {'class':'DDPage__header-place-location'})
    if placeTags is not None :
        df['placeTags'] = placeTags.text
    else :
        df['placeTags'] = ''

    if soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-been-to-top-wrap action-btn-col hidden-print'}) is not None and soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-been-to-top-wrap action-btn-col hidden-print'})[0].find('div', {'class':'title-md item-action-count'}) is not None :
        df['numPeopleVisited'] = int(soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-been-to-top-wrap action-btn-col hidden-print'})[0].find('div', {'class':'title-md item-action-count'}).text)
    else :
        df['numPeopleVisited'] = ''

    if soup.find('div', {'class':'col-xs-4X js-submit-wrap js-like-top-wrap action-btn-col hidden-print'}) is not None and soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-like-top-wrap action-btn-col hidden-print'})[0].find('div', {'class':'title-md item-action-count'}) is not None :
        df['numPeopleWant'] = int(soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-like-top-wrap action-btn-col hidden-print'})[0].find('div', {'class':'title-md item-action-count'}).text)
    else :
        df['numPeopleWant'] = ''

    placeDesc = soup.find('div', {'id':'place-body'})
    if placeDesc is not None :
        df['placeDesc'] = placeDesc.text.replace('\n','')
    else :
        df['placeDesc'] = ''
    
    placeShortDesc = soup.find('h3', {'class': 'DDPage__header-dek'})
    if placeShortDesc is not None :
        df['placeShortDesc'] = placeShortDesc.text.replace('\n','')
    else :
        df['placeShortDesc'] = ''

    placeNearby = set([match.text for match in soup.find_all('div', {'class': 'DDPageSiderailRecirc__item-title'})])
    if len(placeNearby) != 0 :
        df['placeNearby'] = ','.join(list(placeNearby))
    else :
        df['placeNearby'] = ''

    match_address = soup.find_all("address", {"class":"DDPageSiderail__address"})[0]
    if len(match_address) != 0 :
        placeAddress = match_address.find('div').contents[0:5:2]
        df['placeAddress'] = "".join(placeAddress).replace('\n','')
    else :
        df['placeAddress'] = ''

    match = soup.find("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"}).get_text().split()
    if len(match) != 0 :
        df['placeAlt'] = int(round(float(match[0][:-1])))
        df['placeLong'] = int(round(float(match[1])))
    else :
        df['placeAlt'] = ''
        df['placeLong'] = ''

    contributors = []
    for contributor in soup.find_all('div', {'class':'DDPContributors'})[0].find_all('a', {'class':'DDPContributorsList__contributor'}) :
        if contributor.find('span') is not None :
            contributors.append(contributor.find('span').text)
        else :
            contributors.append(contributor.text)
    if len(contributors) != 0 :
        df['placeEditors'] = ','.join(list(set(contributors)))
    else :
        df['placeEditors'] = ''

    placePubDate = soup.find("div", {"class":"DDPContributor__name"})
    if placePubDate is not None :
        df['placePubDate'] = datetime.strptime(placePubDate.text, '%B %d, %Y')
    else :
        df['placePubDate'] = ''

    placeRelatedLists = []
    placeRelatedPlaces = []
    for container in soup.find_all('div', {'class' : 'full-width-container CardRecircSection'}) :
        title = container.find('div', {'class':'CardRecircSection__title'}).text
        if 'Appears' in title :
            for container_related in container.find_all('h3', {'class':'Card__heading --content-card-v2-title js-title-content'}) :
                placeRelatedLists.append(container_related.text[1:-1])
        elif 'Related Places' in title :
            for container_related in container.find_all('h3', {'class':'Card__heading --content-card-v2-title js-title-content'}) :
                placeRelatedPlaces.append(container_related.text[1:-1])
    if len(placeRelatedLists) != 0 :
        df['placeRelatedLists'] = ','.join(placeRelatedLists)
    else :
        df['placeRelatedLists'] = ''
    if len(placeRelatedPlaces) != 0 :
        df['placeRelatedPlaces'] = ','.join(placeRelatedPlaces)
    else :
        df['placeRelatedPlaces'] = ''

    if len(soup.find_all("link", {'rel':'canonical'})) != 0 :
        df['placeURL'] = soup.find_all("link", {'rel':'canonical'})[0]['href']   
    else :
        df['placeURL'] = ''  
        
    return df


In [114]:
file = open(os.getcwd()+'/Locations/page_1/location_1.html', encoding='utf-8')
html_file = file.read()
file.close()

df = get_data_for_one_place(html_file)
df.to_csv(os.getcwd()+'/test.tsv', sep='\t', index=False, header=None)
df


,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"Manhattan, New York",1828,8583,The first New York City subway was built and o...,A beautiful and abandoned New York subway stat...,"African Burial Ground National Monument,Tunnel...","31 Centre StNew York, New York, 10007United St...",41,-74,"erjeffery,jrc2692,Rebekah Otto,offtrackplanet,...",2010-05-08,30 Unexpected Places to Have a Joyful Adventur...,"Crystal Palace Subway,Moscow Metro Stations,Ro...",https://www.atlasobscura.com/places/city-hall-...


---
## **2. Search Engine**

### 2.1. Conjunctive query

 2.1.1) Create your index!

2.1.2) Execute the query

### 2.2. Conjunctive query & Ranking score

2.2.1) Inverted index

2.2.2) Execute the query

---
## **3. Define a new score!**

---
## **4. Visualizing the most relevant places**

---
## **5. BONUS: More complex search engine**

---
## **6. Command line question**

---
## **7. Theoretical question**